In [16]:
import random
from binascii import a2b_hex, b2a_hex

In [17]:
# 1000以内的素数
small_primes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101,
                    103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199,
                    211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317,
                    331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443,
                    449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 571, 577,
                    587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643, 647, 653, 659, 661, 673, 677, 683, 691, 701,
                    709, 719, 727, 733, 739, 743, 751, 757, 761, 769, 773, 787, 797, 809, 811, 821, 823, 827, 829, 839,
                    853, 857, 859, 863, 877, 881, 883, 887, 907, 911, 919, 929, 937, 941, 947, 953, 967, 971, 977, 983,
                    991, 997]

In [18]:
# Miller Rabin算法 判断一个给定的大数是否为素数
def miller_rabin(num, safe_k=1024):
    # num - 1 = 2^s * t
    s = 0
    t = num - 1
    while t % 2 == 0:
        s += 1
        t //= 2
    
    for _ in range(safe_k):
        b = random.randint(2, num - 2)
        r = pow(b, t, num)
        if r == 1 or r == num - 1:
            continue
        for _ in range(s - 1):
            r = pow(r, 2, num)
            if r == num - 1:
                break
        else:
            return False
                
    return True

# 测试
print(miller_rabin(997))
print(miller_rabin(998))

True
False


In [19]:
# 判断一个给定的数是否为素数
def is_prime(num):
    if num < 2: 
        return False
    if num in small_primes:
        return True
    for prime in small_primes:
        if num % prime == 0:
            return False
    return miller_rabin(num)

# 测试
print(is_prime(1009))
print(is_prime(1010))

True
False


In [20]:
# 生成一个指定（二进制）位数的素数
def gen_prime(size=1024):
    while True:
        num = random.randrange(2 ** (size - 1), 2 ** size)
        if is_prime(num):
            return num

# 测试
print(gen_prime(512))

13143602335870395894589676837319383080512065326426019140889379853611636125970594949741812018171876981243398556183701333815674223748621600899892631619838101


In [21]:
# 欧几里得算法（辗转相除） 求两个数的最大公因数
def gcd(a, b):
    remainder = a % b
    while remainder:
        a = b
        b = remainder
        remainder = a % b
    return b

# 测试
print(gcd(3, 27))
print(gcd(911, 999))

3
1


In [22]:
# 拓展欧几里得算法 求模逆元(a ^ -1) % b
def ext_euclid(a, b):
    if b == 0:
        return 1, 0
    else:
        x, y = ext_euclid(b, a % b)
        x, y = y, (x - (a // b) * y)
        return x, y
    
def mod_inverse(a, b):
    return ext_euclid(a, b)[0] % b
# 测试
print(mod_inverse(911, 999))

806


In [23]:
# 快速幂取余算法 求(a ^ b) % n
def fast_exp_mod(a, b, n):
    res = 1
    while b != 0:
        if b & 1:
            res = (res * a) % n
        b >>= 1
        a = pow(a, 2, n)
    return res

# 测试
print(fast_exp_mod(2, 10, 11))
print(fast_exp_mod(11, 15, 13))

1
5


In [24]:
# 写文件
def write_file(content, path):
    with open(path, 'w') as f:
        f.write(content)
    f.close()

In [25]:
# RSA密钥生成
def RSA_KeyGen(size=1024):
    # 生成大素数p q，计算N
    p = 1
    q = 1
    N = p * q
    while N.bit_length() != size:
        p = gen_prime(size // 2)
        q = gen_prime(size - size // 2)
        N = p * q
    # 计算φ(N)
    phi_N = (p - 1)*(q - 1)
    # 选择与φ(N)互素的正整数e
    e = 1
    while True:
        e = random.randrange(3, phi_N)
        if gcd(e, phi_N) == 1:
            break
    # 生成公钥对
    pk = (N, e)
    # 计算d，生成私钥对
    d = mod_inverse(e, phi_N)
    sk = (N, d)

    # 写入文件
    write_file(str(N), 'RSA_Moduler.txt')
    write_file(str(p), 'RSA_p.txt')
    write_file(str(q), 'RSA_q.txt')
    write_file(str(pk), 'RSA_Public_Key.txt')
    write_file(str(sk), 'RSA_Secret_Key.txt')

    return pk, sk

# 测试
print(RSA_KeyGen(1024))

((143408990504848755636628003269322480418088265046452760833397699049329975293521906156150242954590734007093354446207804013475830019340608861971643453970481896745598016985517125409764915607314670169667741890726074098312349604281235954672160102538258164315751330339457035167711827351363746820631898618394418920091, 39257892088264007283706281789340293123404725731181522904488796201630781676384736914959623558694767494207982222490788813049360375538163761675891847952675598871417777061948296452120337884565735644979614033528707621782658730213322360932990623944474990956806515789356063346395611483628257210533123347203948288365), (143408990504848755636628003269322480418088265046452760833397699049329975293521906156150242954590734007093354446207804013475830019340608861971643453970481896745598016985517125409764915607314670169667741890726074098312349604281235954672160102538258164315751330339457035167711827351363746820631898618394418920091, 1430645878631788238874390099134864276680189446543528833802075563

In [26]:
# 加密
def RSA_Enc(pk, m): 
    m_bytes = bytes(m, encoding='utf-8')
    m_hex_str = int(b2a_hex(m_bytes), 16)
    c = fast_exp_mod(m_hex_str, pk[1], pk[0]) # c = (m ^ e) % N
    return c

In [27]:
# 解密
def RSA_Dec(sk, c): 
    m = fast_exp_mod(c, sk[1], sk[0])  # m = (c ^ d) % N
    m_int = a2b_hex(hex(m)[2:])
    m_str = str(m_int, encoding='utf-8')
    return m_str

In [28]:
# 读文件
def read_file(path):
    with open(path, 'r') as f:
        content = f.read()
    f.close()
    return content

In [29]:
# 随机种子
random.seed(520030910281)
# 生成公私钥对
size = 1024
pk, sk = RSA_KeyGen(size)

# 明文消息
plain_msg = read_file('Raw_Message.txt')
print("Raw Message:", plain_msg)

# 加密
cipher_msg = RSA_Enc(pk, plain_msg)
print('Encrypted Message:', hex(cipher_msg))
write_file(hex(cipher_msg), 'Encrypted_Message.txt')

# 解密
decipher_msg = RSA_Dec(sk, cipher_msg)
print("Decrypted Message:", decipher_msg)

Raw Message: Hello RSA!
Encrypted Message: 0x1d94d3d354b81fd295ed3d83ae4a6a1619bec9179925c56a77afeceed0ad598e96175c1ffefe8b6bd1d2d9d7c25b51456380b713daf781e04645a078bdcf18a47c38bdc56d53339089d68ee1834072864583ce6aa8807a93a7b5bb22ced32adb352e1588ccd0d1a2fc193ee1ac7d313a65bc7d541e752701ac6388e5db24a971
Decrypted Message: Hello RSA!
